# HNCDI Explain: Grover Tutorial 3

### Installing qiskit algorithms module

For this tutorial, we will be installing an already implemented Grover's algorithm contained in the qiskit algorithms module

In [ ]:
%pip install 'qiskit[visualization]'==1.4.2
%pip install qiskit-ibm-runtime
%pip install qiskit-aer
%pip install qiskit-algorithms

This is tutorial 3 on Grover's Algorithm. This is a 5-qubit example, with varying Grover iterations. The notebook performs two Grover circuits: the first circuit with 1-iteration and the second with $k'$-iterations.

The code below will do the following. We will first select the maximum number of grover iterations determined by the parameter $k$, where $k-1 = k'$. We will then select a number from a list and then create the corresponding bit-string for the good item. We will import Qiskits existing algorithm that implements Grovers algorithm. We will then run Grover's algorithm for 1-iteration and $k'$-iterations and compare the probability that the marked item was found.

__Task__. In cell 2 below, modify the parameter $k$. How does this change the probability that the marked item is found?


In [ ]:
import numpy as np
from qiskit.visualization import *
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.quantum_info import Statevector
from qiskit_aer.primitives import SamplerV2
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as real_sampler
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile

### Authenticate with IBM quantum

You will require the API token specified in the IBM quantum platform to ensure you can execute your code on a real quantum computer.

In [ ]:
my_token = "<Paste your token here>"

service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)

Define the parameter $k$ that determines the maximum number of Grover iterations.

__Task.__ In the cell below, modify the integer parameter $k$. For example, start with $k$ is $4$ and subsequently increase.

In [ ]:
### Modify Parameter k here ###

k = 4

###

print({'The maximum number of Grover iterations is': k-1})

We will now step through Grover's algorithm, where we will be able to choose the marked item. First, we create a list of numbers from $0$ to $2^n - 1$, where $n$ is the number of qubits. 

In [ ]:
# number of qubits is 5
n = 5
myList = list(range(0, (2**n)))
print(myList)

From this list, choose a number that will correspond to the good marked item. We will convert this to a bit-string. 

Note that this cell also allows you to randomly generate the good marked item. As default, it is set to $31$, the last item on the list.

In [ ]:
# Here we can enter the good item or randomly generate it
secret = 31
#secret = random.randint(0, 2**n -1)  

secret_string = format(secret, "04b")  # format the owner in bit string

print({'The secret number is ': secret})
print({'And the corresponding secret string is ': secret_string})

We will now create the state vector corresponding to the selected bit-string.

In [ ]:
# Create state vector corresponding to the selected string 
oracle = Statevector.from_label(secret_string)  
print('This is the corresponding state for the marked item:')
oracle.draw(output='latex')

We will now import and use the existing Qiskit algorithms that perform Grover's circuit.

In [ ]:
# from qiskit.algorithms import AmplificationProblem
# from qiskit.algorithms import Grover
from qiskit_algorithms.amplitude_amplifiers.amplification_problem import AmplificationProblem
from qiskit_algorithms.amplitude_amplifiers.grover import Grover


problem = AmplificationProblem(oracle, is_good_state=secret_string)

#define several grover circuits for different no. of iterations
grover_circuits = []

print({'Thew maximum number of Grover iterations is': (k-1)})
#Construct Grover circuits for different k
for iteration in range(1, k):
    grover = Grover(iterations=iteration)
    circuit = grover.construct_circuit(problem)
    circuit.measure_all()
    grover_circuits.append(circuit)

We will first submit the grover circuit with 1 iteration to a simulator. 

In [ ]:
# Grover's circuit with 1 iteration
print({'This is the Grover circuit with number of iterations =': (1)})
qc1 = grover_circuits[0]
qc1.draw()

In [ ]:
print({'Recall the good marked item is': secret_string})

sim = AerSimulator()
sampler = SamplerV2()
t_circ = transpile(qc1, sim, optimization_level=3)
results = sampler.run([t_circ], shots=1024).result()
answer = results[0].data.meas.get_counts()
plot_histogram(answer)

We will now submit the grover circuit with $k$ iterations to a simulator. 

In [ ]:
# Grover's circuit with max number of iterations
print({'This is the Grover circuit with number of iterations =': (k-1)})
qcm = grover_circuits[k-2]
qcm.draw()

In [ ]:
print({'Recall the good marked item is': secret_string})

sim = AerSimulator()
sampler = SamplerV2()
t_circ = transpile(qcm, sim, optimization_level=3)
results = sampler.run([t_circ], shots=1024).result()
answer = results[0].data.meas.get_counts()
plot_histogram(answer)

Compare this now to the Grover Circuit with 1-iteration. Has the marked item been found with high probability? How does the number of Grover circuits affect the probability that the marked good item was found? 

In [ ]:
backend = service.least_busy(operational=True, simulator=False)
sampler = real_sampler(backend)
t_circ = transpile(qcm, backend, optimization_level=3)
job = sampler.run([t_circ], shots=1024)
results = job.result()
answer = results[0].data.meas.get_counts()
plot_histogram(answer)